## Урок 4. СУБД MongoDB

1) Модифицировать приложение из предыдущего домашнего задания:
- приложение должно собирать вакансии сразу с двух сайтов hh.ru и superjob.ru 
- собранные данные должны быть приведены к общей структуре
- разделить зарплату на две составляющие (мин. и макс.) и сохранить в виде int. Если валюта указана другая, привести все к рублям. 

In [1]:
from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent
import requests
import re

def get_link(main_link, add_link, key_word):
    link = main_link + add_link + key_word
    return link

def get_html(link, headers = {}):
    if link[0:4] == 'http':
        html = requests.get(link, headers=headers).text
    else:
        with open(link, 'rb') as f:
            html = f.read()
    return html

def parsing_html(link, headers = {}):
    html =  get_html(link, headers=headers)
    parsed_html = bs(html, 'html.parser')
    return parsed_html

def find_class(html, class_):
    found_class = html.findAll(class_=class_)
    return found_class

def find_item(html, class_, href=False):
    found_item = html.find(class_=class_, href=href)
    return found_item

def get_vacancy(html, class_):
    found_class = find_class(html, class_[0])
    vacancy_info = []
    for i in found_class:
        class_dict = {}
        vacancy = find_item(i, class_[1], href=True)
        class_dict['vacancy'] = vacancy.get_text()
        salary = find_item(i, class_[2])
        salary = salary.get_text() if salary else salary
        salary = re.findall('([\d]+)[\s]*([\d]+)', salary) if salary else ['0']
        class_dict['salary'] = [int(''.join(i)) for i in salary] if len(salary) else [0]
        link = vacancy['href']
        link = link if link[0:4]=='http' else main_link + link
        link = re.findall('http[s]?:\/\/[^\?]*', link)
        class_dict['link'] = link[0] if len(link) else ''
        vacancy_info.append(class_dict)
    return vacancy_info


main_link = 'https://hh.ru'
add_link = '/search/vacancy'
key_word = '?text=Программист Разработчик'
class_ = ['vacancy-serp-item', 'bloko-link HH-LinkModifier', 'vacancy-serp-item__compensation']
headers = {'User-Agent': UserAgent().chrome}
link = get_link(main_link, add_link, key_word)
# link = 'hh.html'
parsed_html = parsing_html(link, headers)
vacancy_hh = get_vacancy(parsed_html, class_)
vacancy_hh


main_link = 'https://superjob.ru'
add_link = '/vacancy/search/'
key_word = '?keywords=Программист|Разработчик'
class_ = ['_3syPg', '_1QIBo', '_2Wp8I']
link = get_link(main_link, add_link, key_word)
# link = 'sj.html'
parsed_html = parsing_html(link)
vacancy_sj = get_vacancy(parsed_html, class_)
vacancy_sj


[{'vacancy': 'Web-программист, Web-разработчик',
  'salary': [0],
  'link': 'https://superjob.ru/vakansii/web-programmist-32266222.html'},
 {'vacancy': 'Программист / Разработчик 1С 8.3 УПП',
  'salary': [140000],
  'link': 'https://superjob.ru/vakansii/programmist-32418578.html'},
 {'vacancy': 'Программист / Разработчик',
  'salary': [80000],
  'link': 'https://superjob.ru/vakansii/programmist-32274027.html'},
 {'vacancy': 'Веб-программист, веб-разработчик',
  'salary': [100000],
  'link': 'https://superjob.ru/vakansii/veb-programmist-32441272.html'},
 {'vacancy': 'Программист-разработчик (.NET) на ELMA BPM',
  'salary': [0],
  'link': 'https://superjob.ru/vakansii/programmist-razrabotchik-32408682.html'},
 {'vacancy': 'Программист (разработчик 1С)',
  'salary': [80000],
  'link': 'https://superjob.ru/vakansii/programmist-32221501.html'},
 {'vacancy': 'Программист-разработчик',
  'salary': [60000],
  'link': 'https://superjob.ru/vakansii/programmist-razrabotchik-32382559.html'},
 {'va

2) Реализовать сохранение полученных вакансий в СУБД (на выбор SQLite или MongoDB)

3) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы.

4) \* Модифицировать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта